In [1]:
# 1. 데이터 확보
import zipfile
filepath = 'data/glaucoma.zip'
with zipfile.ZipFile(filepath, 'r') as f:
    f.extractall('data/')

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rotation_range=20, # 최대 회전 각도
    width_shift_range=0.2, # 가로 방향 -20% ~ +20% 이동 정도
    height_shift_range=0.2, # 상하 방향 이동 정도
    horizontal_flip = True, # 수평 뒤집기
    vertical_flip = True # 수직 뒤집기
)
train_data = train_generator.flow_from_directory(
    'data/glaucoma/train',
    target_size=(224,224),
    batch_size=4,
    class_mode='sparse', # 원핫인코딩 안 함
)
test_generator = ImageDataGenerator()
test_data = test_generator.flow_from_directory(
    'data/glaucoma/test',
    target_size=(224,224),
    batch_size=4,
    class_mode='sparse',
)

Found 828 images belonging to 3 classes.
Found 320 images belonging to 3 classes.


In [3]:
# 2. 모델 구성
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
import numpy as np
from tensorflow.keras.models import Sequential # ResNet50의 Dense 층을 사용자 추가
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing import image # 이미지를 읽어오기

resnet_model = ResNet50(input_shape=(224,224,3),
                        include_top=False # 컨볼루션과 풀링 레이어만 구조와 가중치를 가져옴
                       )
resnet_model.trainable = True # ResNet의 가중치도 업데이트 가능
# model = Sequential()
# model.add(resnet_model)
# model.add(Flatten())
# model.add(Dense(units1024, activation='relu'))
# model.add(Dense(units=3, activation='softmax'))
model = Sequential([
    resnet_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()
# 학습 시 필요한 용량
# 1. 파라미터 사용 메모리 사용량 : 파라미터수(126,352,259)*4byte / 1024의 제곱 = 482MB
# 2. 배치사이즈=3일 경우 3* (224*224*3) * 4byte / 1024의 제곱 = 19MB
# 각 layer층 마다 활성화 값

94765736/94765736 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 126,352,259
Trainable params: 126,299,139
Non-trainable params: 53,120
_________________________________________________________________


In [4]:
# 3. 학습 설정
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 4. 학습시키기
model.fit(train_data, validation_data=test_data, epochs=20, batch_size=3)

Epoch 1/20
207/207 [==============================] - 26s 73ms/step - loss: 5.1217 - accuracy: 0.5930 - val_loss: 0.8915 - val_accuracy: 0.6969
Epoch 2/20
207/207 [==============================] - 13s 65ms/step - loss: 0.7544 - accuracy: 0.6570 - val_loss: 6.6907 - val_accuracy: 0.6406
Epoch 3/20
207/207 [==============================] - 13s 65ms/step - loss: 1.1055 - accuracy: 0.6341 - val_loss: 32.9456 - val_accuracy: 0.5063
Epoch 4/20
207/207 [==============================] - 13s 65ms/step - loss: 0.6721 - accuracy: 0.6872 - val_loss: 1.3110 - val_accuracy: 0.6938
Epoch 5/20
207/207 [==============================] - 13s 64ms/step - loss: 0.6477 - accuracy: 0.6908 - val_loss: 2.4556 - val_accuracy: 0.7031
Epoch 6/20
207/207 [==============================] - 13s 64ms/step - loss: 0.6053 - accuracy: 0.7331 - val_loss: 2.1122 - val_accuracy: 0.6906
Epoch 7/20
207/207 [==============================] - 13s 64ms/step - loss: 0.5640 - accuracy: 0.7476 - val_loss: 8.1358 - val_accuracy

In [9]:
# 예측하기
img = image.load_img('data/test.png')
x = image.img_to_array(img).reshape(-1, 224, 224, 3)
pred = model.predict(x)
print(pred)

print(pred.argmax())
import matplotlib.pyplot as plt
plt.figure(figsize=(1,1))
plt.imshow(img)
plt.show()

ValueError: cannot reshape array of size 172800 into shape (224,224,3)